In [25]:
# Importing Libraries
import  pandas as pd 
from sklearn.model_selection import train_test_split
import os


In [26]:
import os
import pandas as pd
folder_path ="C:/Users/user/Desktop/DATASET/airline data for tableau"

complete_airline_data = pd.DataFrame()
for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file:{file_path}")
        temp_df = pd.read_csv(file_path)
        complete_airline_data = pd.concat([complete_airline_data,temp_df], ignore_index = True)

Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_april.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_august.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_dec.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_feb.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_jan.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_july.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_june.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_mar.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_may.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_nov.csv
Processing file:C:/Users/user/Desktop/DATASET/airline data for tableau\airline_oct.csv
Processing file:C:/Users/user/Deskto

In [27]:
complete_airline_data.shape

(6847899, 31)

In [28]:
df=complete_airline_data

In [29]:
# Feature Engineering - creating Key features needed for sampling

df["FlightRoute"] = df["ORIGIN"] + "-" + df["DEST"]
df["AvgDelay"] = df[["DEP_DELAY", "ARR_DELAY"]].mean(axis = 1)
df["CANCELLED"] = df["CANCELLED"].astype(int)

In [30]:
df[["FlightRoute","CANCELLED"]]

,FlightRoute,CANCELLED
0,JFK-BUF,0
1,ITH-JFK,0
2,JFK-ITH,0
3,CLE-JFK,0
4,ATL-FAY,0
...,...,...
6847894,JFK-CLT,0
6847895,DTW-PIT,0
6847896,LGA-RIC,0
6847897,BOS-DCA,0


In [31]:
# Creating Delay and cancellation categories

df["DelayCategory"] = pd.qcut(df["AvgDelay"], q=4, labels=["Low", "Medium", "High", "Severe"])
df["CancellationCategory"] = pd.qcut(df.groupby("FlightRoute")["CANCELLED"].transform("mean"), 
                                             q=4, labels=["Low", "Medium", "High", "Severe"])

In [32]:
print(df[["OP_UNIQUE_CARRIER", "MONTH", "DAY_OF_MONTH", "FlightRoute", "DelayCategory", "CancellationCategory"]].isnull().sum())


OP_UNIQUE_CARRIER           0
MONTH                       0
DAY_OF_MONTH                0
FlightRoute                 0
DelayCategory           84529
CancellationCategory        0
dtype: int64


In [33]:
print(df["DelayCategory"].dtype)


category


In [34]:
df["DelayCategory"] = df["DelayCategory"].astype("category")
df["DelayCategory"] = df["DelayCategory"].cat.add_categories("Unknown")
df["DelayCategory"].fillna("Unknown", inplace=True)


In [35]:
category_counts = df.groupby(
    ["OP_UNIQUE_CARRIER", "MONTH", "DAY_OF_MONTH", "FlightRoute", "DelayCategory", "CancellationCategory"], 
    observed=True
).size()


In [36]:
category_counts = category_counts.reset_index(name="Count")


In [37]:
category_counts = df.groupby(
    ["OP_UNIQUE_CARRIER", "MONTH", "FlightRoute", "DelayCategory"], 
    observed=True
).size()

df["valid_category"] = df.set_index(
    ["OP_UNIQUE_CARRIER", "MONTH", "FlightRoute", "DelayCategory"]
).index.isin(category_counts[category_counts > 1].index)

df_filtered = df[df["valid_category"]].drop(columns=["valid_category"])


In [38]:
# Function to sample the dataset to a million rows
desired_rows = 1_000_000  
sampling_fraction = desired_rows / len(df_filtered)

df_sampled, _ = train_test_split(
    df_filtered, train_size=sampling_fraction, 
    stratify=df_filtered[["OP_UNIQUE_CARRIER", "MONTH", "FlightRoute", "DelayCategory"]], 
    random_state=42
)

In [39]:
df_sampled.shape

(1000000, 35)

In [40]:
df_sampled.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST',
       'DEST_CITY_NAME', 'DEST_STATE_ABR', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'CANCELLATION_CODE', 'DIVERTED', 'AIR_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY',
       'LATE_AIRCRAFT_DELAY', 'FlightRoute', 'AvgDelay', 'DelayCategory',
       'CancellationCategory'],
      dtype='object')

In [41]:
# Compare distributions before and after sampling
print("Airline distribution before sampling:")
print(df_filtered["OP_UNIQUE_CARRIER"].value_counts(normalize=True).head())

print("airline distribution after sampling:")
print(df_sampled["OP_UNIQUE_CARRIER"].value_counts(normalize=True).head())



print("Month distribution before sampling:")
print(df_filtered["MONTH"].value_counts(normalize = True).sort_index())

print("Month distribution before sampling:")
print(df_sampled["MONTH"].value_counts(normalize = True).sort_index())




print("Delay Category distribution before sampling")
print(df_filtered["DelayCategory"].value_counts(normalize=True))

print("Delay Category distribution after sampling")
print(df_sampled["DelayCategory"].value_counts(normalize=True))

Airline distribution before sampling:
OP_UNIQUE_CARRIER
WN    0.210508
DL    0.144437
AA    0.137934
UA    0.107278
OO    0.098491
Name: proportion, dtype: float64
airline distribution after sampling:
OP_UNIQUE_CARRIER
WN    0.211077
DL    0.145136
AA    0.138730
UA    0.107866
OO    0.098708
Name: proportion, dtype: float64
Month distribution before sampling:
MONTH
1     0.078577
2     0.073310
3     0.084727
4     0.081920
5     0.084691
6     0.084250
7     0.087893
8     0.087987
9     0.083154
10    0.087654
11    0.082459
12    0.083378
Name: proportion, dtype: float64
Month distribution before sampling:
MONTH
1     0.078605
2     0.073163
3     0.084791
4     0.081821
5     0.084570
6     0.084032
7     0.087824
8     0.087899
9     0.083265
10    0.087891
11    0.082626
12    0.083513
Name: proportion, dtype: float64
Delay Category distribution before sampling
DelayCategory
High       0.250592
Medium     0.247765
Low        0.247721
Severe     0.243987
Unknown    0.009935
Name:

In [ ]:
output_path = ("C:/Users/user/Desktop/DATASET/airline data for tableau/complete data/sampled_data")
df_sampled.to_csv(output_path, index = False)